In [22]:
import numpy as np
import json
import os
import math
import plotly.graph_objects as go
import plotly.io as pio
import utm
import matplotlib.pyplot as plt

In [2]:
poses_file = '/home/ty/Downloads/AAI/Lane_Marking_Execution/hamburg/odometry/results/2024-06-29_08-31-18/frames_poses.npy'
json_file = '/home/ty/Downloads/AAI/Lane_Marking_Execution/hamburg/data/gnss.json'


In [3]:
def load_states(file):
    data = np.load(file)
    translations = []
    for tf in data:
        translation = tf[:3, 3]
        translations.append(translation)
        
    return data, np.array(translations)

In [4]:
data, translations = load_states(poses_file)

In [9]:
print(type(data))
print(len(data))
print(data[0])
print(data[1])

<class 'numpy.ndarray'>
190
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]
[[ 9.99998907e-01  2.69878043e-04 -1.45347084e-03  1.12953722e+00]
 [-2.69459521e-04  9.99999922e-01  2.88134962e-04  3.61188415e-02]
 [ 1.45354849e-03 -2.87742996e-04  9.99998902e-01 -1.38277067e-02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


In [12]:
print(translations.shape)
print(translations[0])
print(translations[1])

(190, 3)
[0. 0. 0.]
[ 1.12953722  0.03611884 -0.01382771]


In [13]:
def read_gps(file):
    with open(file, 'r') as json_file:
        data = json.load(json_file)
        
    lat = np.array(data['latitude'])
    lon = np.array(data['longitude'])
    alt = np.array(data['altitude'])
    
    data = np.column_stack((lat, lon, alt))
    return data

In [14]:
gps_data = read_gps(json_file)

In [16]:
print(gps_data.shape)
print(gps_data[0])

(76, 3)
[52.4783677 13.3378847 78.106    ]


In [18]:
def convert_global_to_local(gps_data):
    latlon_gps = gps_data[:, :2]
    first_lat, first_lon = latlon_gps[0]
    utm_x, utm_y, zone_num, zone_letter = utm.from_latlon(first_lat, first_lon)
    offset = np.array([utm_x, utm_y])
    utm_mat = []
    for lat, lon in latlon_gps:
        ux, uy, _, _ = utm.from_latlon(lat, lon)
        utm_arr = np.array([ux, uy])
        utm_mat.append(utm_arr)
        
    utm_mat = np.array(utm_mat)
    local_mat = utm_mat - offset
    off_lst = [offset, zone_num, zone_letter]
    
    return local_mat, off_lst

In [19]:
local_gps, offset = convert_global_to_local(gps_data)


In [21]:
print(local_gps.shape)
print(local_gps[0])
print(local_gps[1])

(76, 2)
[0. 0.]
[2.89060501 0.42310164]


In [17]:
def compute_rotation_based_on_gps(gps_data, odom_data, frame_idx=-1, add_offset=0):
    poses_data = odom_data[:, :2]
    gps_data = gps_data[:, :2]
    
    tangent_gps = gps_data[frame_idx] - gps_data[0]
    tangent_lidar = poses_data[frame_idx] - poses_data[0]

    tangent_gps_norm = tangent_gps / np.linalg.norm(tangent_gps)
    tangent_lidar_norm = tangent_lidar / np.linalg.norm(tangent_lidar)

    dot_product = np.dot(tangent_gps_norm, tangent_lidar_norm)
    dot_product = np.clip(dot_product, -1.0, 1.0)
    angle_radians = np.arccos(dot_product)
    sign = np.sign(tangent_gps[0] * tangent_lidar[1] - tangent_gps[1] * tangent_lidar[0])
    angle_radians = sign * angle_radians
    angle_degrees = np.degrees(angle_radians)
    print(angle_degrees)
    angle_degrees = -1*angle_degrees + add_offset
    
    return angle_degrees

In [23]:
angle = compute_rotation_based_on_gps(local_gps, translations,
                                     -1, 0)

-4.475275552324911
